In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# RNN を使ったテキスト分類

In [2]:
# !pip install -q tf-nightly
# import tensorflow_datasets as tfds
# !pip3 list | grep tensorflow
# import tensorflow as tf

# !pip3 install sentencepiece
import sentencepiece as spm

import json
import os
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# GPUを使うために必要
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [3]:
# import matplotlib.pyplot as plt

# def plot_graphs(history, metric):
#   plt.plot(history.history[metric])
#   plt.plot(history.history['val_'+metric], '')
#   plt.xlabel("Epochs")
#   plt.ylabel(metric)
#   plt.legend([metric, 'val_'+metric])
#   plt.show()

## Data


### load

In [4]:
sbj_name = 'Eigo'
attr_name = 'answer_type'

attr_csv_path = f'../{sbj_name}_{attr_name}_ds.tsv'
df = pd.read_csv(attr_csv_path, delimiter='\t')

df

,Unnamed: 0,answer_type,<instruction/>,contents
0,0,(symbol-sentence)*2,次の問い(問１・問２)において，下線部(a)・(b)の単語のアクセント(強勢)の位置が正しい...,"<label>問１</label> <ansColumn id=""A1"">1</ansC..."
1,1,(symbol-sentence)*2,次の問い(問１・問２)において，下線部(a)・(b)の単語のアクセント(強勢)の位置が正しい...,"<label>問２</label> <ansColumn id=""A2"">2</ansC..."
2,2,sentence,次の会話の下線部(1)～(4)について，それぞれ以下の問い(問１～４)に示された①～④のうち...,"<label>問１</label> <ansColumn id=""A3"">3</ansC..."
3,3,sentence,次の会話の下線部(1)～(4)について，それぞれ以下の問い(問１～４)に示された①～④のうち...,"<label>問２</label> <ansColumn id=""A4"">4</ansC..."
4,4,sentence,次の会話の下線部(1)～(4)について，それぞれ以下の問い(問１～４)に示された①～④のうち...,"<label>問３</label> <ansColumn id=""A5"">5</ansC..."
...,...,...,...,...
978,978,sentence,次の問い（問１～５）の 47 ～ 51 に入れるのに最も適当なものを，それぞれ下の①～④のう...,"<label>問２</label> <data id=""D44"" type=""text..."
979,979,sentence,次の問い（問１～５）の 47 ～ 51 に入れるのに最も適当なものを，それぞれ下の①～④のう...,"<label>問３</label> <data id=""D45"" type=""text..."
980,980,sentence,次の問い（問１～５）の 47 ～ 51 に入れるのに最も適当なものを，それぞれ下の①～④のう...,"<label>問４</label> <data id=""D46"" type=""text..."
981,981,sentence,次の問い（問１～５）の 47 ～ 51 に入れるのに最も適当なものを，それぞれ下の①～④のう...,"<label>問５</label> <data id=""D47"" type=""text..."


### Tokenize
SentencePiece を使用。
- タグ あり／なし

In [5]:
from sklearn.model_selection import train_test_split

train_examples, test_examples = train_test_split(
                                    df, test_size=0.2, random_state=0)
train_examples.head(5)

,Unnamed: 0,answer_type,<instruction/>,contents
893,893,sentence,次の問い（問１～４）において，第一アクセント（第一強勢）の位置がほかの三つと異なるものを，そ...,"<label>問２</label> <ansColumn id=""A5"">5</ans..."
158,158,sentence,次の広告に関する以下の問い(問１～３)を読み， 39 ～ 41 に入れるのに最も適当なものを...,"<label>問２</label> <data id=""D30"" type=""text..."
154,154,sentence,次の英文は，体育の授業時間数について，クラスで行われたディスカッションの一部である。文中の ...,<label>Ｃ</label> 次<instruction> の文章の <ref tar...
40,40,sentence,次の問い(問１～５)に対する答えとして最も適当なものを，それぞれ以下の①～④のうちから一つず...,<label>Ｂ</label><instruction>本文の内容と合っているものを，次の...
793,793,sentence,次の問い（問１～５）の 47 ～ 51 に入れるのに最も適当なものを，それぞれ下の①～④のう...,"<label>問５</label><data id=""D44"" type=""text""> W..."


In [6]:
df_tmp = df[['<instruction/>', 'contents']]
df_tmp['<instruction/>'][6]

'次の問い(問１～10)の 7 ～ 16 に入れるのに最も適当なものを，それぞれ以下の①～④のうちから一つずつ選べ。'

In [7]:
m_dir = 'model/SentencePiece'
os.makedirs(m_dir, exist_ok=True)
df.to_csv(f'{m_dir}/tmp.txt', sep='\t')

# arg_str = '--input={m_dir}/tmp.txt --model_prefix={m_dir}/m_user ' + '--user_defined_symbols=<sep>,<cls>' + ',<ansColumn/>,<label>' + ' --vocab_size=2000'
# spm.SentencePieceTrainer.train(arg_str)

spm.SentencePieceTrainer.train(f'--input={m_dir}/tmp.txt --model_prefix={m_dir}/m  --user_defined_symbols=<sep>,<cls>,<pad>   --vocab_size=2000')
sp = spm.SentencePieceProcessor()  # model_file='SentencePiece/test_model.model'

sp.load(f'{m_dir}/m.model')

True

In [8]:
# encode: text => id
tokenized_tokens =  sp.encode_as_pieces('次の問い(問１～３)の会話の 17 ～ 19 に入れるのに最も適当なものを，それぞれ以下の①～④のうちから一つずつ選べ。	')
print(tokenized_tokens)

tokenized_ids = sp.encode_as_ids('次の問い(問１～３)の会話の 17 ～ 19 に入れるのに最も適当なものを，それぞれ以下の①～④のうちから一つずつ選べ。	')
print(tokenized_ids)

decoded_text = sp.decode(tokenized_ids)
print(decoded_text)

['▁次の', '問', 'い', '(', '問', '1～3)', 'の会話の', '▁17', '▁～', '▁19', '▁に入れるのに最も適当な', 'ものを', ',', 'それぞれ以下の', '1～4', 'のうちから一つずつ選', 'べ', '。']
[68, 23, 52, 34, 23, 105, 299, 138, 85, 300, 86, 70, 19, 148, 61, 63, 57, 37]
次の問い(問1～3)の会話の 17 ～ 19 に入れるのに最も適当なものを,それぞれ以下の1～4のうちから一つずつ選べ。


In [9]:
example_content = df_tmp['contents'][20]
print(example_content, sp.encode_as_pieces(example_content))

<label>問２</label><data id="D20" type="text">New information about diet<blank id="B4" />  <ansColumn id="A22">22</ansColumn>  <blank id="B5" />  <ansColumn id="A23">23</ansColumn>  <blank id="B6" /> think is incorrect.</data><choices anscol="A22 A23" comment=""><choice ansnum="1">  <cNum>①</cNum> us</choice><choice ansnum="2">  <cNum>②</cNum> many people</choice><choice ansnum="3">  <cNum>③</cNum> shows</choice><choice ansnum="4">  <cNum>④</cNum> what</choice><choice ansnum="5">  <cNum>⑤</cNum> that</choice> </choices>   ['▁<', 'lab', 'el', '>', '問', '2</', 'lab', 'el', '><', 'data', '▁id', '=', '"', 'D', '20', '"', '▁type', '=', '"', 'text', '"', '>', 'New', '▁information', '▁about', '▁di', 'et', '<', 'blank', '▁id', '=', '"', 'B', '4', '"', '▁/>', '▁<', 'ansColumn', '▁id', '=', '"', 'A', '2', '2', '"', '>', '2', '2</', 'ansColumn', '>', '▁<', 'blank', '▁id', '=', '"', 'B', '5', '"', '▁/>', '▁<', 'ansColumn', '▁id', '=', '"', 'A', '2', '3', '"', '>', '2', '3</', 'ansColumn', '>', '▁<',

In [10]:
# for index in encoded_string:
#   print('{} ----> {}'.format(index, encoder.decode([index])))

## 訓練用データの準備

In [11]:
word2index = {}
# 系列を揃えるためのパディング文字列<pad>を追加
# パディング文字列のIDは0とする
word2index.update({"<pad>":0})

for inst, cont in zip(df['<instruction/>'], df['contents']):
    tokens = sp.encode_as_pieces(inst + cont)
    for word in tokens:
        if word in word2index: continue
        word2index[word] = len(word2index)
print("vocab size : ", len(word2index))


vocab size :  1911


In [12]:
categories = [
    'sentence', 
    'term_person', 'term_location', 'term_time', 'term_other',
    'referenceSymbol',
    'image_graph', 'image_photo', 'image_map', 'image_table', 'image_other',
    'formula', 
    'orthography',
    'other',
    # 組み合わせ系（仮追加）
    '(symbol-sentence)*2', '(symbol-sentence)*3', '(symbol-sentence)*4', '(symbol-term_location)*3', '(symbol-term_other)*3', '(symbol-term_other)*3',
    '(symbol-symbol)*4',
    '(term_location-term_location-term_location)', 'term_location-term_location-term_location-term_location',
    '(term_location-term_location-term_location)', '(term_location-term_location-term_location)',
    '(term_other-term_other-term_other)',
    'sentence-sentence',
    'symbol-symbol-symbol',
    'o(symbol-symbol-symbol-symbol)',
    'o(symbol-symbol-symbol)',
]


### 

In [13]:
## 系列の長さを揃えてバッチでまとめる

from sklearn.model_selection import train_test_split
import random
from sklearn.utils import shuffle

cat2index = {}
for cat in categories:
    if cat in cat2index: continue
    cat2index[cat] = len(cat2index)

def sentence2index(sentence):
    tokens = sp.encode_as_pieces(sentence)
    # print(tokens)
    return [word2index[w] for w in tokens]

def category2index(cat):
    return [cat2index[cat]]

index_datasets_c_xml_tmp = []
index_datasets_category = []

# 系列の長さの最大値を取得。この長さに他の系列の長さをあわせる
max_len = 0
for inst, cont, category in zip(df['<instruction/>'], df['contents'], df['answer_type']):
  index_c_xml = sentence2index(inst + cont)
  index_category = category2index(category)
  index_datasets_c_xml_tmp.append(index_c_xml)
  index_datasets_category.append(index_category)
  if max_len < len(index_c_xml):
    max_len = len(index_c_xml)

# 系列の長さを揃えるために短い系列にパディングを追加
# 後ろパディングだと正しく学習できなかったので、前パディング
index_datasets_c_xml = []
for c_xml in index_datasets_c_xml_tmp:
  for i in range(max_len - len(c_xml)):
    c_xml.insert(0, 0) # 前パディング
#     c_xml.append(0)　# 後ろパディング
  index_datasets_c_xml.append(c_xml)

train_x, test_x, train_y, test_y = train_test_split(index_datasets_c_xml, index_datasets_category, train_size=0.7)

# データをバッチでまとめるための関数
def train2batch(c_xml, category, batch_size=100):
  c_xml_batch = []
  category_batch = []
  c_xml_shuffle, category_shuffle = shuffle(c_xml, category)
  for i in range(0, len(c_xml), batch_size):
    c_xml_batch.append(c_xml_shuffle[i:i+batch_size])
    category_batch.append(category_shuffle[i:i+batch_size])
  return c_xml_batch, category_batch

## モデルの作成

In [14]:
class LSTMClassifier(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        # <pad>の単語IDが0なので、padding_idx=0としている
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        # batch_first=Trueが大事！
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.softmax = nn.LogSoftmax()

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        #embeds.size() = (batch_size × len(sentence) × embedding_dim)
        _, lstm_out = self.lstm(embeds)
        # lstm_out[0].size() = (1 × batch_size × hidden_dim)
        tag_space = self.hidden2tag(lstm_out[0])
        # tag_space.size() = (1 × batch_size × tagset_size)

        # (batch_size × tagset_size)にするためにsqueeze()する
        tag_scores = self.softmax(tag_space.squeeze())
        # tag_scores.size() = (batch_size × tagset_size)

        return tag_scores

# 単語の埋め込み次元数上げた。精度がそこそこアップ！ハイパーパラメータのチューニング大事。
EMBEDDING_DIM = 200
HIDDEN_DIM = 128
VOCAB_SIZE = len(word2index)
TAG_SIZE = len(categories)
# to(device)でモデルがGPU対応する
model = LSTMClassifier(EMBEDDING_DIM, HIDDEN_DIM, VOCAB_SIZE, TAG_SIZE).to(device)
loss_function = nn.NLLLoss()
# SGDからAdamに変更。特に意味はなし
optimizer = optim.Adam(model.parameters(), lr=0.001)

## モデルの訓練

In [16]:
losses = []
for epoch in range(100):
    all_loss = 0
    title_batch, category_batch = train2batch(train_x, train_y)
    for i in range(len(title_batch)):
        batch_loss = 0

        model.zero_grad()

        # 順伝搬させるtensorはGPUで処理させるためdevice=にGPUをセット
        title_tensor = torch.tensor(title_batch[i], device=device)
        # category_tensor.size() = (batch_size × 1)なので、squeeze()
        category_tensor = torch.tensor(category_batch[i], device=device).squeeze()

        out = model(title_tensor)

        batch_loss = loss_function(out, category_tensor)
        batch_loss.backward()
        optimizer.step()

        all_loss += batch_loss.item()
    print("epoch", epoch, "\t" , "loss", all_loss)
    if all_loss < 0.1: break
print("done.")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


epoch 0 	 loss 18.353562116622925
epoch 1 	 loss 8.046313345432281
epoch 2 	 loss 6.411768734455109
epoch 3 	 loss 6.1079630851745605
epoch 4 	 loss 5.906205236911774
epoch 5 	 loss 5.816978633403778
epoch 6 	 loss 5.777347922325134
epoch 7 	 loss 5.721486747264862
epoch 8 	 loss 5.629653513431549
epoch 9 	 loss 5.535482347011566
epoch 10 	 loss 5.455973744392395
epoch 11 	 loss 5.290098488330841
epoch 12 	 loss 5.068092346191406
epoch 13 	 loss 4.7663169503211975
epoch 14 	 loss 4.224766790866852
epoch 15 	 loss 3.7899158000946045
epoch 16 	 loss 3.5029635429382324
epoch 17 	 loss 3.160395711660385
epoch 18 	 loss 3.0177665054798126
epoch 19 	 loss 2.7753783762454987
epoch 20 	 loss 2.4795882552862167
epoch 21 	 loss 2.1993174701929092
epoch 22 	 loss 2.0467266142368317
epoch 23 	 loss 1.85104900598526
epoch 24 	 loss 1.5724025443196297
epoch 25 	 loss 1.4392117112874985
epoch 26 	 loss 1.372746929526329
epoch 27 	 loss 1.242957390844822
epoch 28 	 loss 1.079236850142479
epoch 29 	 lo

In [17]:
test_num = len(test_x)
a = 0
with torch.no_grad():
    title_batch, category_batch = train2batch(test_x, test_y)

    for i in range(len(title_batch)):
        title_tensor = torch.tensor(title_batch[i], device=device)
        category_tensor = torch.tensor(category_batch[i], device=device)

        out = model(title_tensor)
        _, predicts = torch.max(out, 1)
        for j, ans in enumerate(category_tensor):
            if predicts[j].item() == ans.item():
                a += 1
print("predict : ", a / test_num)
# predict :  0.6967916854948034

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


predict :  0.864406779661017


## 2つ以上の LSTM レイヤーを重ねる

Keras のリカレントレイヤーには、コンストラクタの `return_sequences` 引数でコントロールされる2つのモードがあります。

* それぞれのタイムステップの連続した出力のシーケンス全体（shape が `(batch_size, timesteps, output_features)` の3階テンソル）を返す。
* それぞれの入力シーケンスの最後の出力だけ（shape が `(batch_size, output_features)` の2階テンソル）を返す。

In [18]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])

NameError: name 'tf' is not defined

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset,
                    validation_steps=30)

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

In [ ]:
# パディングなしのサンプルテキストの推論

sample_pred_text = ('The movie was not good. The animation and the graphics '
                    'were terrible. I would not recommend this movie.')
predictions = sample_predict(sample_pred_text, pad=False)
print(predictions)

In [ ]:
# パディングありのサンプルテキストの推論

sample_pred_text = ('The movie was not good. The animation and the graphics '
                    'were terrible. I would not recommend this movie.')
predictions = sample_predict(sample_pred_text, pad=True)
print(predictions)

In [ ]:
plot_graphs(history, 'accuracy')

In [ ]:
plot_graphs(history, 'loss')

[GRU レイヤー](https://www.tensorflow.org/api_docs/python/tf/keras/layers/GRU)など既存のほかのレイヤーを調べてみましょう。

カスタム RNN の構築に興味があるのであれば、[Keras RNN ガイド](../../guide/keras/rnn.ipynb) を参照してください。